<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [20]:
# Import libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string
import pickle

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Dense

# <a name="lectura"> 2. Lectura del dataframe <a>


In [21]:
#Import parquet file.

tweets_df = pd.read_parquet('../../data/processed/tweets.parquet')

# Show the head of the dataframe.

tweets_df.head(10)

,text,sentiment,SentimentText_clean
0,id have responded if i were going,0,id responded going
1,sooo sad i will miss you here in san diego,2,sooo sad miss san diego
2,my boss is bullying me,2,boss bullying
3,what interview leave me alone,2,interview leave alone
4,sons of why couldnt they put them on the rel...,2,sons couldnt put releases already bought
5,some shameless plugging for the best rangers...,0,shameless plugging best rangers forum earth
6,2am feedings for the baby are fun when he is a...,1,2am feedings baby fun smiles coos
7,soooo high,0,soooo high
8,both of you,0,
9,journey wow u just became cooler hehe is tha...,1,journey wow u became cooler hehe possible


In [59]:
# Save in tweets the column "SentimentText_clean" and in sentimientos the column "sentiment".   

tweets = tweets_df["SentimentText_clean"]

sentimientos = tweets_df["sentiment"].values

In [61]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



# Dividir los datos en características y etiquetas
X = tweets
y = tweets_df['sentiment'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Crear el Tokenizer y ajustarlo solo a las palabras del conjunto de entrenamiento
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convertir las secuencias de palabras en secuencias numéricas
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Aplicar padding para que todas las secuencias tengan la misma longitud
max_sequence_length = max([len(sequence) for sequence in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Convertir las etiquetas a one-hot encoding
num_classes = 3
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes)

# Crear el modelo de redes neuronales
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train_padded, y_train_encoded, validation_data=(X_test_padded, y_test_encoded), epochs=10, batch_size=32)


Epoch 1/10
687/687 [==============================] - 77s 105ms/step - loss: 0.8052 - accuracy: 0.6380 - val_loss: 0.6909 - val_accuracy: 0.7088
Epoch 2/10
687/687 [==============================] - 72s 105ms/step - loss: 0.5135 - accuracy: 0.7959 - val_loss: 0.7345 - val_accuracy: 0.6980
Epoch 3/10
687/687 [==============================] - 73s 106ms/step - loss: 0.3260 - accuracy: 0.8790 - val_loss: 0.8553 - val_accuracy: 0.6880
Epoch 4/10
687/687 [==============================] - 75s 109ms/step - loss: 0.2148 - accuracy: 0.9236 - val_loss: 1.0571 - val_accuracy: 0.6798
Epoch 5/10
687/687 [==============================] - 74s 107ms/step - loss: 0.1432 - accuracy: 0.9499 - val_loss: 1.2096 - val_accuracy: 0.6604
Epoch 6/10
687/687 [==============================] - 73s 107ms/step - loss: 0.1090 - accuracy: 0.9627 - val_loss: 1.4571 - val_accuracy: 0.6764
Epoch 7/10
687/687 [==============================] - 74s 108ms/step - loss: 0.0878 - accuracy: 0.9687 - val_loss: 1.6090 - val_ac

In [81]:
model.save("../../models/nn_tweets.h5")

In [82]:
max_sequence_length

17

In [ ]:
input_text = pd.Series(request.form['text'])[0]

In [74]:
# Predict the sentiment of a tweet.

def predict_sentiment(text):

    # Convert the text to sequence of words
    X_test_sequences = tokenizer.texts_to_sequences([text])

    # Pad the sequence
    X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

    # Make predictions
    predictions = model.predict(X_test_padded)

    # Get the sentiment with the highest probability
    sentiment = np.argmax(predictions)

    # Put name to the sentiment

    if sentiment == 0:
        sentiment = 'Neutral',
    elif sentiment == 1:
        sentiment = 'Positive'
    else:
        sentiment = 'Negative'


    return sentiment

In [80]:
predict_sentiment("I hate my girlfriend")

1/1 [==============================] - 0s 46ms/step


'Negative'

In [62]:
# Save the tokenizer in models.

with open('../../models/tweets_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [37]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])


In [39]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Entrenar el modelo
model.fit(padded_sequences, sentimientos, epochs=10, validation_split=0.2)

Epoch 1/10
687/687 [==============================] - 43s 61ms/step - loss: 0.7896 - accuracy: 0.6477 - val_loss: 0.6829 - val_accuracy: 0.7229
Epoch 2/10
687/687 [==============================] - 43s 62ms/step - loss: 0.5522 - accuracy: 0.7805 - val_loss: 0.7106 - val_accuracy: 0.7020
Epoch 3/10
687/687 [==============================] - 42s 61ms/step - loss: 0.3523 - accuracy: 0.8730 - val_loss: 0.8570 - val_accuracy: 0.6846
Epoch 4/10
687/687 [==============================] - 43s 63ms/step - loss: 0.1924 - accuracy: 0.9351 - val_loss: 1.0696 - val_accuracy: 0.6605
Epoch 5/10
687/687 [==============================] - 42s 62ms/step - loss: 0.1054 - accuracy: 0.9662 - val_loss: 1.2942 - val_accuracy: 0.6520
Epoch 6/10
687/687 [==============================] - 43s 63ms/step - loss: 0.0640 - accuracy: 0.9804 - val_loss: 1.6097 - val_accuracy: 0.6498
Epoch 7/10
687/687 [==============================] - 44s 64ms/step - loss: 0.0450 - accuracy: 0.9870 - val_loss: 1.8108 - val_accuracy:

In [24]:
# Crear el modelo de la red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.MaxPooling1D(5),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(64, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entrenar el modelo con validación cruzada
history = model.fit(padded_sequences, sentimientos, epochs=20, validation_split=0.2, batch_size=32)



Epoch 1/20
687/687 [==============================] - 22s 29ms/step - loss: 1.0151 - accuracy: 0.4689 - val_loss: 0.7697 - val_accuracy: 0.6631
Epoch 2/20
687/687 [==============================] - 5s 7ms/step - loss: 0.7289 - accuracy: 0.6988 - val_loss: 0.6758 - val_accuracy: 0.7178
Epoch 3/20
687/687 [==============================] - 4s 6ms/step - loss: 0.5802 - accuracy: 0.7719 - val_loss: 0.7030 - val_accuracy: 0.7142
Epoch 4/20
687/687 [==============================] - 4s 7ms/step - loss: 0.4708 - accuracy: 0.8212 - val_loss: 0.7474 - val_accuracy: 0.7033
Epoch 5/20
687/687 [==============================] - 4s 6ms/step - loss: 0.3847 - accuracy: 0.8555 - val_loss: 0.8447 - val_accuracy: 0.7022
Epoch 6/20
687/687 [==============================] - 3s 5ms/step - loss: 0.3166 - accuracy: 0.8860 - val_loss: 0.9472 - val_accuracy: 0.6875
Epoch 7/20
687/687 [==============================] - 5s 7ms/step - loss: 0.2697 - accuracy: 0.9041 - val_loss: 1.0619 - val_accuracy: 0.6789
Epoc

In [28]:
# Crear el modelo de la red neuronal
model = tf.keras.Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    MaxPooling1D(5),
    Dropout(0.5),
    Conv1D(64, 5, activation="relu"),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(3, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entrenar el modelo con validación cruzada
history = model.fit(padded_sequences, sentimientos, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
687/687 [==============================] - 8s 12ms/step - loss: 1.0202 - accuracy: 0.4670 - val_loss: 0.8594 - val_accuracy: 0.5701
Epoch 2/20
687/687 [==============================] - 8s 12ms/step - loss: 0.8210 - accuracy: 0.6325 - val_loss: 0.7230 - val_accuracy: 0.7004
Epoch 3/20
687/687 [==============================] - 8s 12ms/step - loss: 0.6926 - accuracy: 0.7123 - val_loss: 0.6940 - val_accuracy: 0.7097
Epoch 4/20
687/687 [==============================] - 8s 11ms/step - loss: 0.6116 - accuracy: 0.7555 - val_loss: 0.7083 - val_accuracy: 0.6911
Epoch 5/20
687/687 [==============================] - 8s 12ms/step - loss: 0.5469 - accuracy: 0.7890 - val_loss: 0.7112 - val_accuracy: 0.7097
Epoch 6/20
687/687 [==============================] - 8s 12ms/step - loss: 0.5012 - accuracy: 0.8103 - val_loss: 0.7398 - val_accuracy: 0.7011
Epoch 7/20
687/687 [==============================] - 8s 12ms/step - loss: 0.4609 - accuracy: 0.8271 - val_loss: 0.7676 - val_accuracy: 0.6957

In [63]:
model.save("../../models/nn_tweets2.h5")

In [67]:
# Load the model of the neural network.

model = tf.keras.models.load_model('../../models/nn_tweets2.h5')

# Load the tokenizer.

# Input text.

text = input("Ingrese el texto: ")

with open('../../models/tweets_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Convertir las secuencias de palabras en secuencias numéricas
text = tokenizer.texts_to_sequences(text)

# Aplicar padding para que todas las secuencias tengan la misma longitud
max_sequence_length = len(text)
text = pad_sequences(text, maxlen=25, padding='post')

prediction = model.predict(text)

print(prediction)



1/1 [==============================] - 1s 1s/step
[[9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [4.15356159e-01 3.13416566e-03 5.81509709e-01]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.87786174e-01 5.32803312e-03 6.88591553e-03]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.95299578e-01 8.20325731e-05 4.61843098e-03]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.99979019e-01 1.52212615e-05 5.67355437e-06]
 [9.97592211e-01 1.61777937e-03 7.90042512e-04]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530846e-02]
 [9.80702698e-01 6.14428660e-03 1.31530836e-02]]
